In [14]:
import numpy as np
import pandas as pd 

In [16]:
import utils

In [17]:
train, test, info, index = utils.ReadData("train_big")

In [18]:


# I'm gong to share a solution that I found interesting with you.
# The idea is to compute the median of the series in different window sizes at the end of the series,
# and the window sizes are increasing exponentially with the base of golden ratio.
# Then a median of these medians is taken as the estimate for the next 60 days.
# This code's result has the score of around 44.9 on public leaderboard, but I could get upto 44.7 by playing with it.

# r = 1.61803398875
# Windows = np.round(r**np.arange(0,9) * 7)
Windows = [6, 12, 18, 30, 48, 78, 126, 203, 329]

In [19]:
from tqdm import tqdm

In [20]:
n = train.shape[1] - 1 #  550
Visits = np.zeros(train.shape[0])

for i, row in tqdm(train.iterrows(), total=train.shape[0]):
    M = []
    start = row[1:].nonzero()[0]
    if len(start) == 0:
        continue
    if n - start[0] < Windows[0]:
        Visits[i] = row.iloc[start[0]+1:].median()
        continue
    for W in Windows:
        if W > n-start[0]:
            break
        M.append(row.iloc[-W:].median())
    Visits[i] = np.median(M)

Visits[np.where(Visits < 1)] = 0.
train['Visits'] = Visits

  0%|          | 1/145063 [00:01<54:00:30,  1.34s/it]/home/shirobokov/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:3250: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
100%|██████████| 145063/145063 [03:51<00:00, 627.12it/s]
/opt/conda/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in less


In [25]:
y_pred = np.tile(train['Visits'].values, (60, 1)).T

In [27]:
y_pred = np.nan_to_num(y_pred)

In [29]:
utils.SMAPE_score(test, y_pred, print_mean=True)

39.818916


array([32.874947, 38.364967, 59.185104, ..., 84.70818 , 67.58499 ,
       69.719925], dtype=float32)

In [30]:
utils.SaveModel("fibonachi_medians",
                y_pred, test,
                240, "fill with zeros",
                index.values.reshape(-1))

39.818916
